# Example: Overview of Primary Query Functionality

### Imports and Token Assignment

In [1]:
import os
from pprint import pprint

token = os.environ['EC3_KEY']

### Querying EPDs

In [ ]:
#Getting EPD records example
from ec3 import EC3epds
ec3_epds = EC3epds(bearer_token=token, ssl_verify=False)

epd_param_dict = {
                "product_classes":{"EC3":"Concrete >> ReadyMix"}, 
                "concrete_compressive_strength_28d":"3000 psi", 
                "lightweight":True, 
                "applicable_in":["NAFTA"]
                }

ec3_epds.max_records = 10 #default max records is 100
ec3_epds.only_valid = False #Default is True. You can set this to False if you want to also return expired EPDs
epd_records = ec3_epds.get_epds(params=epd_param_dict)

pprint(epd_records[0])

In [ ]:
#Get EPDs while using masterformat filter (A similar approach using the 'masterformat_filter' property will work with the EC3Materials class)
#(In this example we return all rebar EPDs)
from ec3 import EC3epds
ec3_epds2 = EC3epds(bearer_token=token, ssl_verify=False)
ec3_epds2.only_valid = True
ec3_epds2.masterformat_filter = ["03 21 00 Reinforcement Bars"]
epd_param_dict2 = {}

epd_records2 = ec3_epds2.get_epds(params=epd_param_dict2)

pprint(epd_records2[0])

In [ ]:
#Get an epd by xpd uuid
epd_xpd_uuid = "ec3ftbxgp9"

epd_record = ec3_epds.get_epd_by_xpduuid(epd_xpd_uuid)
pprint(epd_record)

### Querying Materials

In [ ]:
#Getting material records example
from ec3 import EC3Materials
ec3_materials = EC3Materials(bearer_token=token, ssl_verify=False)

#Dictonary of params and values. See EC3 Api documentation for options.
mat_param_dict = {"lightweight":False, "concrete_compressive_strength_at_28d__target":"3000 psi", "jurisdiction":"US-NY"}

ec3_materials.return_fields = ["id", "concrete_compressive_strength_28d", "gwp"] #This can be used to limit the fields beings returned
ec3_materials.sort_by = "concrete_compressive_strength_28d" #This will sort the responses based on the field assiged to the 'sort_by' property

#NOTE The following query may take a few minutes to return all responses depending on parameters passed
#Setting return_all to True will ignore the max_records number and attempt to return all matches
mat_records = ec3_materials.get_materials(return_all=True, params=mat_param_dict)

pprint(mat_records[0])

In [ ]:
#Get materials within a given distance of a postal code
#Conduct a search of normal weights concrete mixes between 2000 psi and 8000 psi within a radius of a given postal code
mat_param_dict =   {
                    "product_classes":{"EC3":"Concrete >> ReadyMix"},
                    "lightweight":False, 
                    "concrete_compressive_strength_at_28d__gt":"2000 psi", 
                    "concrete_compressive_strength_at_28d__lt":"8000 psi", 
                    }

ec3_materials.return_fields = ["id", "concrete_compressive_strength_28d", "gwp", "plant_or_group"]
ec3_materials.sort_by = "concrete_compressive_strength_28d" #This will sort the responses based on the field assiged to the 'sort_by' property
ec3_materials.only_valid = True
ec3_materials.max_records = 50
postal_code = 11232

mat_records = ec3_materials.get_materials_within_region(postal_code, plant_distance="10 mi", return_all=False, params=mat_param_dict)
pprint(mat_records[0])

### Retrieving Projects

In [ ]:
#Retrieve a project examples. 
#You will need projects setup in your EC3 account or be looking at publicly available projects for this to work
from ec3 import EC3Projects
ec3_projects = EC3Projects(bearer_token=token, ssl_verify=False)

#Search by name
project_name = "[PROJECT NAME TO SEARCH]"
proj_by_name = ec3_projects.get_projects_by_name(project_name)

#Search by project id
project_id = "fd0e951748f44dceb9d51c3c6a298ea2"
proj_by_id = ec3_projects.get_project_by_id(project_id)

pprint(proj_by_id[0])

### Retrieving Categories

In [ ]:
#Get category by id
from ec3 import EC3Categories
ec3_categories = EC3Categories(bearer_token=token, ssl_verify=False)

cat_by_id = ec3_categories.get_category_by_id('05d49f750eed4648b7f72e3bbfc8b8e4')
pprint(cat_by_id)

In [ ]:
#Get root category with the entire tree
whole_tree = ec3_categories.get_all_categories()

#Once you have the entire tree you can use some of the functions in utils for crawlingthrough the tree
#The following function will create a dictionary with masterformat codes as keys and ids as values
from ec3 import ec3_utils
test_dict = ec3_utils.get_masterformat_category_dict(whole_tree)
pprint(test_dict)